In [9]:
import numpy as np
import pymc3 as pm

# Data Generation

In [2]:
def generate_X1(size=100, random_state=123):
    rand = np.random.RandomState(random_state)
    mu = rand.normal(loc=10, scale=3, size=size)
    x1 = [rand.normal(loc=m, scale=4) for m in mu]
    return np.array(x1)


def generate_X2(size=100, random_state=123):
    rand = np.random.RandomState(random_state)
    mu = rand.normal(loc=10, scale=3, size=size)
    sigma = rand.uniform(2, 6, size=size)
    x2 = [rand.normal(loc=m, scale=s) for m, s in zip(mu, sigma)]
    return np.array(x2)

In [3]:
x1 = generate_X1()
x2 = generate_X2()

# Models

## Model 1

In [11]:
with pm.Model() as model_x1_m1:
    mu = pm.Uniform('mu', lower=0, upper=20)
    obs = pm.Normal('obs', mu=mu, sigma=5, observed=x1)


with pm.Model() as model_x2_m1:
    mu = pm.Uniform('mu', lower=0, upper=20)
    obs = pm.Normal('obs', mu=mu, sigma=5, observed=x2)

## Model 2

In [16]:
with pm.Model() as model_x1_m2:
    mu = pm.Uniform('mu', lower=0, upper=20)
    sigma = pm.Uniform('sigma', lower=1, upper=10)
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=x1)
    
with pm.Model() as model_x2_m2:
    mu = pm.Uniform('mu', lower=0, upper=20)
    sigma = pm.Uniform('sigma', lower=1, upper=10)
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=x2)

# Calculation of Bayes factors using sequential MC